# Ensembl genes table extraction EDA

This notebook is useful for development as well as exploratory data analysis on the extracted tables.
It is currently automically executed and saved as part of exports using `papermill`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from bioregistry import normalize_prefix

from ensembl_genes import ensembl_genes

In [3]:
# parameters cell
species = "human"
release = "111"

In [4]:
# Parameters
species = "human"
release = "115"


In [5]:
ensg = ensembl_genes.Ensembl_Gene_Queries(release=release, species=species)
ensg.connection_url

'mysql+mysqlconnector://anonymous@ensembldb.ensembl.org:3306/homo_sapiens_core_115_38'

In [6]:
database = ensg.database
database

'homo_sapiens_core_115_38'

## Extract data

## gene attrib counts

In [7]:
ensg.run_query("gene_attrib_counts").head(15)

,attrib_type_id,code,name,description,attrib_type_count,attrib_type_examples
0,4,name,Name,Alternative/long name,84546,"RP11-314A20.1, RP11-190J23.3, RP11-815I9.3, EN..."
1,395,xref_id,Xref ID,ID of associated database reference,56019,"OTTHUMG00000157657, OTTHUMG00000153133, OTTHUM..."
2,538,legacy_biotype,Legacy biotype,Obsolete biotype previously assigned to this E...,36490,"lincrna, antisense, processed_transcript, sens..."
3,54,remark,Remark,Annotation remark,20799,full approved gene name is:ST7 overlapping tra...
4,536,Ensembl_Select,Ensembl Select,The Ensembl Select is a transcript identified ...,19338,"ENST00000440869, ENST00000304749, ENST00000366..."
5,380,havana_cv,Havana CV term,Controlled vocabulary terms from Havana,19218,"overlapping locus, reference genome error, rea..."
6,142,GeneGC,Gene GC,Percentage GC content for this gene,37,"48.61, 46.28, 39.61, 46.20, 42.67, 33.82, 40.1..."
7,560,artef_dupl,Artifactual duplication,Annotation on artifactual regions of the genom...,19,Artifactual duplication. Real copy of this gen...
8,382,NoTransRefError,No translations due to reference error,This gene is believed to include protein codin...,10,1
9,1,embl_acc,European Nucleotide Archive (was EMBL) accession,ENA,0,None


## genes

In [8]:
ensg.gene_df.head()

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
0,ENSG00000000003,17,TSPAN6,HGNC,HGNC:11858,protein_coding,ensembl_havana,2008-04-29 11:17:41,2025-02-22 09:10:37,GRCh38,...,100627108,100639991,-1,True,NaN,no,tetraspanin 6,HGNC Symbol,HGNC:11858,ENSG00000000003
1,ENSG00000000005,6,TNMD,HGNC,HGNC:17757,protein_coding,ensembl_havana,2008-04-29 11:17:41,2018-11-21 17:23:49,GRCh38,...,100584936,100599885,1,True,NaN,no,tenomodulin,HGNC Symbol,HGNC:17757,ENSG00000000005
2,ENSG00000000419,15,DPM1,HGNC,HGNC:3005,protein_coding,ensembl_havana,2008-04-29 11:17:41,2025-02-22 09:10:37,GRCh38,...,50934852,50959140,-1,True,NaN,no,dolichyl-phosphate mannosyltransferase subunit...,HGNC Symbol,HGNC:3005,ENSG00000000419
3,ENSG00000000457,15,SCYL3,HGNC,HGNC:19285,protein_coding,ensembl_havana,2008-04-29 11:17:41,2025-02-22 09:10:37,GRCh38,...,169846981,169894267,-1,True,NaN,no,SCY1 like pseudokinase 3,HGNC Symbol,HGNC:19285,ENSG00000000457
4,ENSG00000000460,18,FIRRM,HGNC,HGNC:25565,protein_coding,ensembl_havana,2008-04-29 11:17:41,2025-02-22 09:10:37,GRCh38,...,169662007,169855456,1,True,NaN,no,FIGNL1 interacting regulator of recombination ...,HGNC Symbol,HGNC:25565,ENSG00000000460


In [9]:
# clone-based genes no longer get a symbol and are filled with the stable ID
# https://www.ensembl.info/2021/03/15/retirement-of-clone-based-gene-names/
ensg.gene_df.query("gene_symbol == ensembl_gene_id").head(2)

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
1598,ENSG00000083622,8,ENSG00000083622,None,None,lncRNA,havana,2008-04-29 11:17:41,2009-05-19 09:47:17,GRCh38,...,117604791,117647415,-1,True,NaN,no,"novel transcript, antisense to CFTR",NaN,NaN,ENSG00000083622
1995,ENSG00000093100,13,ENSG00000093100,None,None,lncRNA,havana,2008-04-29 11:17:41,2012-06-07 23:07:01,GRCh38,...,17787652,17811497,-1,True,NaN,no,novel transcript,NaN,NaN,ENSG00000093100


In [10]:
# which external database the gene symbol derives from versus the ensembl source
pd.crosstab(
    ensg.gene_df.ensembl_source,
    ensg.gene_df.gene_symbol_source_db.fillna("missing (clone-based)"),
    margins=True,
)

gene_symbol_source_db,EntrezGene,HGNC,RFAM,miRBase,missing (clone-based),All
ensembl_source,,,,,,
ensembl,0,4214,1469,0,848,6531
ensembl_havana,28,21636,1,0,276,21941
ensembl_havana_tagene,0,7,0,0,1,8
havana,263,18782,0,2,16942,35989
havana_tagene,14,326,0,0,19578,19918
insdc,0,37,0,0,0,37
mirbase,0,1918,0,1,26,1945
All,305,46920,1470,3,37671,86369


In [11]:
ensg.gene_df.coord_system.value_counts().head(10)

chromosome    78691
scaffold       7678
Name: coord_system, dtype: int64

In [12]:
ensg.gene_df.gene_biotype.value_counts().head(10)

lncRNA                                36490
protein_coding                        23262
processed_pseudogene                  10234
unprocessed_pseudogene                 2802
misc_RNA                               2418
snRNA                                  2094
miRNA                                  1945
transcribed_unprocessed_pseudogene     1801
transcribed_processed_pseudogene       1219
TEC                                    1084
Name: gene_biotype, dtype: int64

In [13]:
pd.crosstab(ensg.gene_df.coord_system, ensg.gene_df.primary_assembly, margins=True)

primary_assembly,False,True,All
coord_system,,,
chromosome,0,78691,78691
scaffold,7678,0,7678
All,7678,78691,86369


In [14]:
ensg.gene_df.mhc.value_counts()

no      85641
MHC       494
xMHC      234
Name: mhc, dtype: int64

In [15]:
len(ensg.gene_df)

86369

## alternative gene alleles

Related:

- [OTP: Origin of genes_with_non_reference_ensembl_ids.tsv](https://github.com/opentargets/platform/issues/702)
- [biostars: map between different assemblies of one ensembl release](https://www.biostars.org/p/143956/)
- using `attrib_type.code = "non_ref"` for `primary_assembly` doesn't appear to return any results

In [16]:
ensg.representative_gene_df.head()

,rs_allele_group,ensembl_gene_id,gene_symbol,ensembl_created_date,seq_region,primary_assembly,alt_allele_group_id,alt_allele_attrib,alt_allele_is_representative,ensembl_representative_gene_id,is_representative_gene
0,44430,ENSG00000273644,None,2014-06-09 10:49:07,7,True,44430.0,IS_REPRESENTATIVE,True,ENSG00000273644,True
2,44430,ENSG00000282333,None,2015-06-01 18:57:05,HSCHR7_2_CTG1,False,44430.0,AUTOMATICALLY_ASSIGNED,False,ENSG00000273644,False
3,44431,ENSG00000281993,None,2015-06-01 18:57:05,HSCHR7_1_CTG1,False,44431.0,AUTOMATICALLY_ASSIGNED,False,ENSG00000281993,True
4,44431,ENSG00000282645,None,2015-06-01 18:57:05,HSCHR7_2_CTG1,False,44431.0,AUTOMATICALLY_ASSIGNED,False,ENSG00000281993,False
5,44431,ENSG00000288372,None,2019-06-15 05:41:31,HG1309_PATCH,False,44431.0,AUTOMATICALLY_ASSIGNED,False,ENSG00000281993,False


In [17]:
# looks like non_ref isn't set for human genes
query = '''
SELECT *
FROM gene_attrib
LEFT JOIN attrib_type
  ON gene_attrib.attrib_type_id = attrib_type.attrib_type_id
WHERE attrib_type.code = "non_ref"
LIMIT 5
'''
pd.read_sql(sql=query, con=ensg.connection_url)

,gene_id,attrib_type_id,value,attrib_type_id,code,name,description


In [18]:
ensg.representative_gene_df.alt_allele_attrib.value_counts()

AUTOMATICALLY_ASSIGNED    6469
IS_REPRESENTATIVE         3905
IS_PAR                      79
MANUALLY_ASSIGNED           74
Name: alt_allele_attrib, dtype: int64

In [19]:
ensg.representative_gene_df.query("ensembl_gene_id != ensembl_representative_gene_id").head(2)

,rs_allele_group,ensembl_gene_id,gene_symbol,ensembl_created_date,seq_region,primary_assembly,alt_allele_group_id,alt_allele_attrib,alt_allele_is_representative,ensembl_representative_gene_id,is_representative_gene
2,44430,ENSG00000282333,None,2015-06-01 18:57:05,HSCHR7_2_CTG1,False,44430.0,AUTOMATICALLY_ASSIGNED,False,ENSG00000273644,False
4,44431,ENSG00000282645,None,2015-06-01 18:57:05,HSCHR7_2_CTG1,False,44431.0,AUTOMATICALLY_ASSIGNED,False,ENSG00000281993,False


# replaced ID converter

A single `old_stable_id` can map to multiple `new_stable_id`. For example, `ENSG00000152006`

https://uswest.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=AzhM62SpkvdiLC4H-6808613

Requested ID | Matched ID(s) | Releases
-- | -- | --
ENSG00000152006 | ENSG00000196273 | 26: ENSG00000196273.1
ENSG00000152006 | ENSG00000197016 | 26: ENSG00000197016.1
ENSG00000152006 | ENSG00000196239 | 26: ENSG00000196239.1

In [20]:
ensg.old_to_new_df.head(2)

,old_ensembl_gene_id,new_ensembl_gene_id
0,ENSG00000011319,ENSG00000187391
1,ENSG00000077754,ENSG00000197226


In [21]:
# some ensembl genes replaced by many new ensembl genes
ensg.old_to_new_df.old_ensembl_gene_id.value_counts().head(2)

ENSG00000201456    91
ENSG00000193147    90
Name: old_ensembl_gene_id, dtype: int64

In [22]:
# example
ensg._update_ensembl_gene("ENSG00000152006")

{'ENSG00000196239', 'ENSG00000196273', 'ENSG00000197016'}

In [23]:
ensg.old_to_newest_df.head(2)

,old_ensembl_gene_id,newest_ensembl_gene_id,is_current
0,ASMPATCHG00000000170,ENSG00000256229,True
1,ASMPATCHG00000000174,ENSG00000188171,True


In [24]:
len(ensg.old_to_newest_df)

20486

In [25]:
ensg.old_to_newest_df.is_current.value_counts()

True     11447
False     9039
Name: is_current, dtype: int64

## omni-updater

The omni-updater dataset is designed to convert ensembl gene IDs from input data to the current, representative ensembl_gene_ids for this ensembl release. It assumes:

- users want to update outdated genes with their replacements
- users want a dataset of representative genes only, and want to convert alternative alleles to representative genes

An inner join of a dataset with `update_df` on `input_ensembl_gene_id` will do the following:

- produce output ensembl_gene_ids that are current and representatives
- update outdated genes with their current identifiers. Outdated genes with no current replacement will be removed by the inner join.
- update alternative gene alleles with their representatives
- genes that are already represenative and current will map to themselves

In [26]:
ensg.update_df.head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
0,ASMPATCHG00000000170,ENSG00000256229,False,True,1,3
2,ASMPATCHG00000000174,ENSG00000188171,False,True,1,3


In [27]:
ensg.update_df.sort_values("input_maps_to_n_genes", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
5391,ENSG00000201456,ENSG00000206922,False,True,90,5
3593,ENSG00000201456,ENSG00000206767,False,True,90,6


In [28]:
ensg.update_df.sort_values("n_inputs_map_to_gene", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
20946,ENSG00000207105,ENSG00000202251,True,False,1,914
18628,ENSG00000200547,ENSG00000202251,True,False,1,914


In [29]:
(ensg.update_df.input_maps_to_n_genes == 1).mean()

0.9438690451381653

In [30]:
ensg.update_df.query("ensembl_gene_id == 'ENSG00000256263'")

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
636,ASMPATCHG00000001513,ENSG00000256263,False,True,1,13
44495,ENSG00000256263,ENSG00000256263,True,True,1,13
637,ENSG00000260270,ENSG00000256263,False,True,1,13
638,ENSG00000262195,ENSG00000256263,False,True,1,13
639,ENSG00000263289,ENSG00000256263,False,True,1,13
640,ENSG00000269129,ENSG00000256263,False,True,1,13
641,ENSG00000269829,ENSG00000256263,False,True,1,13
642,ENSG00000270320,ENSG00000256263,False,True,1,13
643,ENSG00000271301,ENSG00000256263,False,True,1,13
644,ENSG00000271747,ENSG00000256263,False,True,1,13


In [31]:
print(
    f"The omni-updater contains {len(ensg.update_df):,} rows for mapping "
    f"{ensg.update_df.input_ensembl_gene_id.nunique():,} input genes to "
    f"{ensg.update_df.ensembl_gene_id.nunique():,} current, representative genes."
)

The omni-updater contains 95,972 rows for mapping 91,342 input genes to 77,981 current, representative genes.


In [32]:
# https://useast.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=P45VLMbogubpI0QA-6815464
ensg.update_df.query("input_ensembl_gene_id == 'ENSG00000201456'").head(3)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
5704,ENSG00000201456,ENSG00000199482,False,True,90,2
3472,ENSG00000201456,ENSG00000199601,False,True,90,7
5705,ENSG00000201456,ENSG00000199664,False,True,90,3


## cross-refrences (xrefs)

In [33]:
ensg.xref_df.head()

,ensembl_representative_gene_id,ensembl_gene_id,gene_symbol,xref_source,xref_accession,xref_label,xref_description,xref_info_type,xref_linkage_annotation,xref_curie
0,ENSG00000000003,ENSG00000000003,TSPAN6,ArrayExpress,ENSG00000000003,ENSG00000000003,None,DIRECT,None,arrayexpress:ENSG00000000003
1,ENSG00000000003,ENSG00000000003,TSPAN6,EntrezGene,7105,TSPAN6,tetraspanin 6,DEPENDENT,None,ncbigene:7105
2,ENSG00000000003,ENSG00000000003,TSPAN6,GeneCards,11858,TSPAN6,tetraspanin 6,DEPENDENT,None,genecards:11858
3,ENSG00000000003,ENSG00000000003,TSPAN6,HGNC,HGNC:11858,TSPAN6,tetraspanin 6,DIRECT,None,hgnc:11858
4,ENSG00000000003,ENSG00000000003,TSPAN6,MIM_GENE,300191,TETRASPANIN 6; TSPAN6 [*300191],TETRASPANIN 6; TSPAN6;;TRANSMEMBRANE 4 SUPERFA...,DEPENDENT,None,omim:300191


In [34]:
# datasets where there are ensembl_gene_id-xref_source-xref_accession pairs might not be distinct
xref_dup_df = ensg.xref_df[ensg.xref_df.duplicated(subset=["ensembl_gene_id", "xref_source", "xref_accession"], keep=False)]
xref_dup_df.xref_source.value_counts()

Series([], Name: xref_source, dtype: int64)

In [35]:
# xref sources versus info_types
df = pd.crosstab(ensg.xref_df.xref_source, ensg.xref_df.xref_info_type, margins=True)
df["bioregistry_prefix"] = df.index.to_series().replace(ensg._xref_prefix_updater).map(normalize_prefix)
df

xref_info_type,DEPENDENT,DIRECT,MISC,All,bioregistry_prefix
xref_source,,,,,
ArrayExpress,0,86369,0,86369,arrayexpress
DBASS3,0,19,0,19,None
DBASS5,0,14,0,14,None
ENS_LRG_gene,0,1324,0,1324,lrg
EntrezGene,36775,0,0,36775,ncbigene
GeneCards,45195,0,0,45195,genecards
HGNC,1727,45195,0,46922,hgnc
MIM_GENE,19248,0,0,19248,omim
MIM_MORBID,8205,0,0,8205,omim


## Gene Ontology xrefs

In [36]:
ensg.xref_go_df.head(3)

,ensembl_gene_id,go_id,go_label,go_evidence_codes,xref_info_types,xref_info_texts,ensembl_transcript_ids,ensembl_representative_gene_id
0,ENSG00000000003,GO:0005515,protein binding,IPI,DIRECT,UniProt,ENST00000373020,ENSG00000000003
1,ENSG00000000003,GO:0005886,plasma membrane,IBA,DIRECT,GO_Central,ENST00000373020,ENSG00000000003
2,ENSG00000000003,GO:0016020,membrane,IEA,"DEPENDENT,DIRECT",",InterPro,UniProt","ENST00000373020,ENST00000612152,ENST0000086788...",ENSG00000000003


In [37]:
# GO terms for CCR5
# compare to http://useast.ensembl.org/Homo_sapiens/Gene/Ontologies/molecular_function?g=ENSG00000160791
sorted(ensg.xref_go_df.query("ensembl_gene_id == 'ENSG00000160791'").go_label.dropna())

['C-C chemokine binding',
 'C-C chemokine receptor activity',
 'G protein-coupled receptor activity',
 'G protein-coupled receptor signaling pathway',
 'MAPK cascade',
 'actin binding',
 'apoptotic process',
 'calcium ion transport',
 'calcium-mediated signaling',
 'cell chemotaxis',
 'cell surface receptor signaling pathway',
 'cell-cell signaling',
 'cellular defense response',
 'cellular response to cytokine stimulus',
 'cellular response to lipopolysaccharide',
 'chemokine (C-C motif) ligand 5 binding',
 'chemokine receptor activity',
 'chemokine-mediated signaling pathway',
 'chemotaxis',
 'coreceptor activity',
 'cytoplasm',
 'defense response',
 'endosome',
 'identical protein binding',
 'immune response',
 'inflammatory response',
 'membrane',
 'negative regulation of macrophage apoptotic process',
 'phosphatidylinositol phospholipase C activity',
 'plasma membrane',
 'positive regulation of cytosolic calcium ion concentration',
 'protein binding',
 'release of sequestered calc

## lrg xrefs

In [38]:
ensg.xref_lrg_df.head(2)

,ensembl_gene_id,lrg_gene_id
103,ENSG00000000971,LRG_47
158,ENSG00000001084,LRG_1166


In [39]:
len(ensg.xref_lrg_df)

1324

### ncbigene xrefs

In [40]:
ensg.xref_ncbigene_df.head()

,ensembl_representative_gene_id,ncbigene_id,gene_symbol,ncbigene_symbol
1,ENSG00000000003,7105,TSPAN6,TSPAN6
9,ENSG00000000005,64102,TNMD,TNMD
16,ENSG00000000419,8813,DPM1,DPM1
45,ENSG00000000457,57147,SCYL3,SCYL3
53,ENSG00000000460,55732,FIRRM,FIRRM


In [41]:
# ensembl gene mapped to by multiple ncbigenes
ensg.xref_ncbigene_df.ensembl_representative_gene_id.value_counts().head(3)

ENSG00000000003    1
ENSG00000207721    1
ENSG00000207717    1
Name: ensembl_representative_gene_id, dtype: int64

In [42]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ensembl_representative_gene_id.duplicated().sum()

(27578, 0)

In [43]:
# ncbigene mapped to by multiple ensembl genes, likely due to alt gene alleles
ensg.xref_ncbigene_df.ncbigene_id.value_counts().head(3)

124906683    9
9437         7
102724956    5
Name: ncbigene_id, dtype: int64

In [44]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ncbigene_id.duplicated().sum()

(27578, 84)

In [45]:
# ensg.xref_ncbigene_df.query("ensembl_representative_gene_id == 'ENSG00000231500'")
# ensg.xref_ncbigene_df.query("ncbigene_id == '51206'")

In [46]:
repr_ensembl_gene_ids = set(ensg.gene_df.ensembl_representative_gene_id)
len(repr_ensembl_gene_ids)

77981

In [47]:
# many of these genes should probably be alternative alleles rather than representative
ensg.gene_df.query("not primary_assembly and ensembl_gene_id==ensembl_representative_gene_id")

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
16975,ENSG00000196101,10,HLA-DRB3,HGNC,HGNC:4951,protein_coding,ensembl_havana,2008-04-29 11:17:41,2023-04-14 17:13:51,GRCh38,...,3715355,3728422,-1,False,NaN,no,"major histocompatibility complex, class II, DR...",HGNC Symbol,HGNC:4951,ENSG00000196101
23319,ENSG00000215523,7,ENSG00000215523,None,None,lncRNA,havana,2007-09-07 00:01:32,2015-06-01 18:57:05,GRCh38,...,2053740,2085765,-1,False,NaN,no,chromosome 6 open reading frame 214 (putative),NaN,NaN,ENSG00000215523
24768,ENSG00000223359,2,ENSG00000223359,None,None,unprocessed_pseudogene,havana,2009-05-19 09:47:17,2010-03-18 16:07:21,GRCh38,...,3789961,3790056,-1,False,NaN,no,"major histocompatibility complex, class II, DR...",NaN,NaN,ENSG00000223359
24907,ENSG00000223557,1,ENSG00000223557,None,None,lncRNA,havana,2009-05-19 09:47:17,2009-05-19 09:47:17,GRCh38,...,3361186,3361479,1,False,NaN,no,novel transcript,NaN,NaN,ENSG00000223557
25127,ENSG00000223844,1,ENSG00000223844,None,None,processed_pseudogene,havana,2009-05-19 09:47:17,2009-05-19 09:47:17,GRCh38,...,3948261,3949391,1,False,NaN,no,"family with sequence similarity 8, member A5 p...",NaN,NaN,ENSG00000223844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86069,ENSG00000310277,1,ENSG00000310277,None,None,lncRNA,havana_tagene,2024-04-29 16:53:40,2024-04-29 16:53:40,GRCh38,...,67572,97360,-1,False,NaN,no,novel transcript,NaN,NaN,ENSG00000310277
86139,ENSG00000310347,1,ENSG00000310347,None,None,lncRNA,havana_tagene,2024-04-29 16:53:40,2024-04-29 16:53:40,GRCh38,...,6600,8034,1,False,NaN,no,novel transcript,NaN,NaN,ENSG00000310347
86168,ENSG00000310376,1,ENSG00000310376,None,None,lncRNA,havana_tagene,2024-04-29 16:53:40,2024-04-29 16:53:40,GRCh38,...,113837,140657,1,False,NaN,no,novel transcript,NaN,NaN,ENSG00000310376
86184,ENSG00000310392,1,ENSG00000310392,None,None,lncRNA,havana_tagene,2024-04-29 16:53:40,2024-04-29 16:53:40,GRCh38,...,120506,121345,-1,False,NaN,no,novel transcript,NaN,NaN,ENSG00000310392
